---
# Install the required packages

If needed install the following packages:

In [1]:
# !pip install datasets transformers imbalanced-learn evaluate

---
# Imports

In [2]:
import numpy as np
import pandas as pd

from datasets import Dataset, load_dataset

from transformers import DistilBertForSequenceClassification
from transformers import DistilBertTokenizer
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding

from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB


import evaluate
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

C:\Users\Nikola\AppData\Roaming\Python\Python310\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
C:\Users\Nikola\AppData\Roaming\Python\Python310\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
C:\Users\Nikola\AppData\Roaming\Python\Python310\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


---
# Laboratory Exercise - Run Mode (8 points)

## Introduction

This laboratory assignment's primary objective is to fine-tune a pre-trained language model for detection of toxic sentences (binary classification). 

The dataset contains two attributes: 
- `text`: The sentence which needs to be classified in to toxic/non-toxic
- `label`: 0/1 indicator if the given sentence is toxic

**Note: You are required to perform this laboratory assignment on your local machine.**

# Read the data

The dataset reading is given. Just run the following 2 cells.

**DO NOT MODIFY IT! Just analyse how the data reading was performed, as in the future this part won't be given.**

In [3]:
dataset = load_dataset(
    'csv', 
    data_files={'train': 'data/train.tsv', 'val': 'data/val.tsv','test': 'data/test.tsv'},
    delimiter='\t'
)

**The prediction target column MUST be named 'label' in the dataset !**

See the dataset structure:

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 3130
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3132
    })
})

---
# Natural Language Processing

## Generate the Tokenizer and Data Collator

For the purposes of this lab you will be using `DistilBertTokenizer` and `DataCollatorWithPadding`.

In [5]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Tokenize the dataset

For the purposes of lowering the amount of computing set the `max_length` parameter to 15.

In [6]:
dataset = dataset.map(lambda x: tokenizer(x['text'], truncation=True, padding=True, max_length=15), batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3130 [00:00<?, ? examples/s]

Map:   0%|          | 0/3132 [00:00<?, ? examples/s]

## Define the model

The required model for this lab is the `DistilBertForSequenceClassification`.

In [7]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Define the training arguments

For lowering the compute time I recommend using the following parameters:
- per_device_train_batch_size=128
- per_device_eval_batch_size=128
- **num_train_epochs=1**

In [8]:
training_args = TrainingArguments(
    output_dir="trainer",
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    metric_for_best_model="f1",
    num_train_epochs=1
)

## Load the metrics

Load the best metric for the this specific problem.

In [9]:
metric = evaluate.load('f1')

### Define the function to compute the metrics

In [10]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

## Generate the Trainer object

In [11]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['val'],
    compute_metrics=compute_metrics,
    data_collator=collator,
)

## Train the model

Use the trainer to train the model.

In [12]:
trainer.train()

  0%|          | 0/8 [00:00<?, ?it/s]

{'train_runtime': 214.9979, 'train_samples_per_second': 4.651, 'train_steps_per_second': 0.037, 'train_loss': 0.6441447138786316, 'epoch': 1.0}


TrainOutput(global_step=8, training_loss=0.6441447138786316, metrics={'train_runtime': 214.9979, 'train_samples_per_second': 4.651, 'train_steps_per_second': 0.037, 'train_loss': 0.6441447138786316, 'epoch': 1.0})

---
# Evaluate the model

## Generate predictions for the test set

In [13]:
predictions = trainer.predict(dataset['test'])

  0%|          | 0/25 [00:00<?, ?it/s]

## Extract the predictions (class 0 or 1) from the logits

In [14]:
preds = np.argmax(predictions.predictions, axis=-1)

## Analyze the performance of the model

In [15]:
print(classification_report(dataset['test']['label'], preds))

              precision    recall  f1-score   support

           0       0.84      0.72      0.78      1566
           1       0.76      0.86      0.80      1566

    accuracy                           0.79      3132
   macro avg       0.80      0.79      0.79      3132
weighted avg       0.80      0.79      0.79      3132



# Laboratory Exercise - Bonus Task (+ 2 points)

Implement a simple machine learning pipeline to classify if a given text is **toxic** or not. Use TF-IDF vectorization to convert text into numerical features and train a `MultinomialNB` model. If needed use `RandomUnderSampler()`. Compare the results with the transformer model.

In [16]:
pipeline = make_pipeline(
    TfidfVectorizer(),
    RandomUnderSampler(),
    MultinomialNB()
)

In [17]:
pipeline.fit(dataset['train']['text'], dataset['train']['label'])

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('randomundersampler', RandomUnderSampler()),
                ('multinomialnb', MultinomialNB())])

In [18]:
preds_nb = pipeline.predict(dataset['test']['text'])

In [19]:
print(classification_report(dataset['test']['label'], preds_nb))

              precision    recall  f1-score   support

           0       0.84      0.71      0.77      1566
           1       0.75      0.87      0.81      1566

    accuracy                           0.79      3132
   macro avg       0.80      0.79      0.79      3132
weighted avg       0.80      0.79      0.79      3132



---